# ......................................................BLAKE2b-512....................................................

In [1]:
import numpy as np
import warnings as w
w.simplefilter('ignore')

## constants 

In [2]:
IV = np.array([0x6A09E667F3BCC908,
               0xBB67AE8584CAA73B,
               0x3C6EF372FE94F82B,
               0xA54FF53A5F1D36F1,
               0x510E527FADE682D1,
               0x9B05688C2B3E6C1F,
               0x1F83D9ABFB41BD6B,
               0x5BE0CD19137E2179],dtype="uint64")

In [3]:
SIGMA = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
         [14, 10, 4, 8, 9, 15, 13, 6, 1, 12, 0, 2, 11, 7, 5, 3],
         [11, 8, 12, 0, 5, 2, 15, 13, 10, 14, 3, 6, 7, 1, 9, 4],
         [7, 9, 3, 1, 13, 12, 11, 14, 2, 6, 5, 10, 4, 0, 15, 8],
         [9, 0, 5, 7, 2, 4, 10, 15, 14, 1, 11, 12, 6, 8, 3, 13],
         [2, 12, 6, 10, 0, 11, 8, 3, 4, 13, 7, 5, 15, 14, 1, 9],
         [12, 5, 1, 15, 14, 13, 4, 10, 0, 7, 6, 3, 9, 2, 8, 11],
         [13, 11, 7, 14, 12, 1, 3, 9, 5, 0, 15, 4, 8, 6, 2, 10],
         [6, 15, 14, 9, 11, 3, 0, 8, 12, 2, 13, 7, 1, 4, 10, 5],
         [10, 2, 8, 4, 7, 6, 1, 5, 15, 11, 9, 14, 3, 12, 13, 0]]

## Blake2b function

In [4]:
def blake2b_512(m):
    cBytesCompressed = 0
    cBytesRemaining  = len(m)
    counter=0
    h = np.array(IV,dtype="uint64")
    p = np.array(0x01010040,dtype="uint64")
    h[0] = h[0]^p
    while(cBytesRemaining > 256):
        chunk = getMessage(m,counter)
        cBytesCompressed = cBytesCompressed + 256 # increase count of bytes that have been compressed
        cBytesRemaining  = cBytesRemaining  - 256  #decrease count of bytes in M remaining to be processed
        h = compress(h, chunk, cBytesCompressed, False)  #false ⇒ this is not the last chunk
        counter+=256
        
    chunk = getMessage(m,counter)
    cBytesCompressed = cBytesCompressed+cBytesRemaining  #The actual number of bytes leftover in M
    chunk = pad(chunk, 256)  #If M was empty, then we will still compress a final chunk of zeros
    h = compress(h, chunk, cBytesCompressed, True)  #true ⇒ this is the last chunk
    bhash = finalEndian(h)
    return bhash

In [5]:
def finalEndian(h):
    ha=[]
    blakehash=""
    for i in range (0,8,1):
        ha.append(hex(h[i]))
        ha[i]=ha[i][2:]
        if(len(ha[i])!=16):
            ha[i]="0"*(16-len(ha[i]))+ha[i]
    for i in range(0,len(ha),1):
        ba = bytearray.fromhex(ha[i])
        ba.reverse()
        ha[i] = ''.join(format(x,'02x')for x in ba).upper()
        blakehash=blakehash+ha[i]
    return blakehash

In [6]:
def convertToHex(m):
    return m.encode("UTF-8").hex().upper()

In [7]:
def getMessage(m,counter):
    return m[counter:counter+256]

In [8]:
def pad(m,size):
    m=convertToHex(m);
    for i in range(len(m),size,1):
        if(len(m)<size):
            m+='0'
    return m

### compress function

In [9]:
def compress(h,chunk,t,isLast):
    f = np.uint64(0xFFFFFFFFFFFFFFFF)
    v=np.array(h,dtype="uint64")
    v= np.append(v,IV)
    v=np.uint64(v)
    t0,t1 = getCounter(t)
    v[12] = v[12]^t0
    v[13] = v[13]^t1
    chunk = littleEndian(chunk)
    msg = np.uint64(chunk)
    if isLast:
        v[14] = v[14]^f
    for i in range(0,12,1):
        S = SIGMA[i % 10]
        #Column step
        v[0], v[4], v[8],  v[12] = G(v[0], v[4], v[8],  v[12], msg[S[0]], msg[S[1]])
        v[1], v[5], v[9],  v[13] = G(v[1], v[5], v[9],  v[13], msg[S[2]], msg[S[3]])
        v[2], v[6], v[10], v[14] = G(v[2], v[6], v[10], v[14], msg[S[4]], msg[S[5]])
        v[3], v[7], v[11], v[15] = G(v[3], v[7], v[11], v[15], msg[S[6]], msg[S[7]])
        
        #Diagonal step
        v[0], v[5], v[10], v[15] = G(v[0], v[5], v[10], v[15], msg[S[8]],  msg[S[9]])
        v[1], v[6], v[11], v[12] = G(v[1], v[6], v[11], v[12], msg[S[10]], msg[S[11]])
        v[2], v[7], v[8],  v[13] = G(v[2], v[7], v[8],  v[13], msg[S[12]], msg[S[13]])
        v[3], v[4], v[9],  v[14] = G(v[3], v[4], v[9],  v[14], msg[S[14]], msg[S[15]])
    for i in range(0,8,1):
        h[i] = h[i]^v[i]^v[i+8]
    return h

In [10]:
def getCounter(t):
    t = bin(t)
    t = "0"+t[2:]
    for i in range(len(t),128,1):
        t = "0"+t
    t1 = np.uint64(int(t[0:64],2))
    t0 = np.uint64(int(t[64:128],2))
    return t0,t1

In [11]:
def littleEndian(m):
    msg = [m[i:i+16]for i in range(0,len(m),16)]
    for i in range(0,len(msg),1):
        ba = bytearray.fromhex(msg[i])
        ba.reverse()
        msg[i] = ''.join(format(x,'02x')for x in ba)
        msg[i] = int(msg[i],16)
    return msg

### G-function 

In [12]:
#G function
def G(a,b,c,d,x,y):
    r1=np.uint64(32)
    r2=np.uint64(24)
    r3=np.uint64(16)
    r4=np.uint64(63)
    a = a+b+x
    d = ROR64((d^a),r1)
    c = c+d
    b = ROR64((b^c),r2)
    a = a+b+y
    d = ROR64((d^a),r3)
    c = c+d
    b = ROR64((b^c),r4)
    return a,b,c,d

In [13]:
#Rotate Right 
def ROR64(x,y):
    r = np.uint64(64)
    return (((x) >> (y)) ^ ((x) << (r - (y))))

## Outputs

In [14]:
blake2b_512("abc")

'BA80A53F981C4D0D6A2797B69F12F6E94C212F14685AC4B74B12BB6FDBFFA2D17D87C5392AAB792DC252D5DE4533CC9518D38AA8DBF1925AB92386EDD4009923'

In [15]:
blake2b_512("")

'786A02F742015903C6C6FD852552D272912F4740E15847618A86E217F71F5419D25E1031AFEE585313896444934EB04B903A685B1448B755D56F701AFE9BE2CE'

In [16]:
blake2b_512("The quick brown fox jumps over the lazy dog")

'A8ADD4BDDDFD93E4877D2746E62817B116364A1FA7BC148D95090BC7333B3673F82401CF7AA2E4CB1ECD90296E3F14CB5413F8ED77BE73045B13914CDCD6A918'

In [17]:
Einp = "abc"
Eoup = "BA80A53F981C4D0D6A2797B69F12F6E94C212F14685AC4B74B12BB6FDBFFA2D17D87C5392AAB792DC252D5DE4533CC9518D38AA8DBF1925AB92386EDD4009923"
Ainp = "abc"
Aoup = "BA80A53F981C4D0D6A2797B69F12F6E94C212F14685AC4B74B12BB6FDBFFA2D17D87C5392AAB792DC252D5DE4533CC9518D38AA8DBF1925AB92386EDD4009923"
Eoup==Aoup

True

In [18]:
Einp1 = ""
Eoup1 = "786A02F742015903C6C6FD852552D272912F4740E15847618A86E217F71F5419D25E1031AFEE585313896444934EB04B903A685B1448B755D56F701AFE9BE2CE"
Ainp1 = ""
Aoup1 = "786A02F742015903C6C6FD852552D272912F4740E15847618A86E217F71F5419D25E1031AFEE585313896444934EB04B903A685B1448B755D56F701AFE9BE2CE"
Eoup1==Aoup1

True

In [19]:
Einp2 = "The quick brown fox jumps over the lazy dog"
Eoup2 = "A8ADD4BDDDFD93E4877D2746E62817B116364A1FA7BC148D95090BC7333B3673F82401CF7AA2E4CB1ECD90296E3F14CB5413F8ED77BE73045B13914CDCD6A918"
Ainp2 = "The quick brown fox jumps over the lazy dog"
Aoup2 = "A8ADD4BDDDFD93E4877D2746E62817B116364A1FA7BC148D95090BC7333B3673F82401CF7AA2E4CB1ECD90296E3F14CB5413F8ED77BE73045B13914CDCD6A918"
Eoup2==Aoup2

True